In [ ]:
import json
import math
import pathlib
import collections

import pandas as pd
from matplotlib import pyplot as plt

from wikidbs.colors import COLOR_9A

pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [ ]:
path = pathlib.Path("../data/stats.json")
with open(path, "r", encoding="utf-8") as file:
    stats = json.load(file)

In [ ]:
print("num databases:", len(stats["num_tables"]))
ss = []
for key in ("num_tables", "num_cols", "num_rows", "sparsities", "num_numerical", "num_non_numerical"):
    s = pd.Series(stats[key], name=key).describe()
    s["sum"] = sum(stats[key])
    del s["count"]
    ss.append(s)
pd.concat(ss, axis=1)

In [ ]:
print(list(sorted(stats["sparsities"], reverse=True))[:10])
print(len(list(filter(lambda x: x > 0.6, stats["sparsities"]))))

In [ ]:
print("num unique table names:", len(stats["tab_names"].keys()))
print("num unique column names:", len(stats["col_names"].keys()))
print("num unique llm table names:", len(stats["llm_tab_names"].keys()))
print("num unique llm column names:", len(stats["llm_col_names"].keys()))

In [ ]:
print("num total table names:", sum(stats["tab_names"].values()))
print("num total column names:", sum(stats["col_names"].values()))
print("num total llm table names:", sum(stats["llm_tab_names"].values()))
print("num total llm column names:", sum(stats["llm_col_names"].values()))

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (11, 1.5)
plt.rcParams["font.size"] = 14
figure, (tabs, cols, rows) = plt.subplots(nrows=1, ncols=3, sharex=False, sharey=False)
figure.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.55, hspace=0.7)

tabs.hist(stats["num_tables"], bins=list(range(0, 11, 1)), color="#e74c3c")
tabs.set_xlabel("# tables")
tabs.set_xticks((0, 2.5, 5, 7.5, 10), labels=("0", "", "5", "", "10"))
tabs.set_xlim((0, 10))
tabs.set_ylabel("# databases")
tabs.set_yticks((0, 5000, 10000), labels=("0", "", "10k"))
tabs.set_ylim((0, 10000))

cols.hist(stats["num_cols"], bins=list(range(0, 101, 10)), color="#e74c3c")
cols.set_xlabel("# columns")
cols.set_xticks((0, 25, 50, 75, 100), labels=("0", "", "50", "", "100"))
cols.set_xlim((0, 100))
cols.set_ylabel("# tables")
cols.set_yticks((0, 40000, 80000, 120000), labels=("0", "", "", "120k"))
cols.set_ylim((0, 120000))

rows.hist(stats["num_rows"], bins=list(range(0, 251, 25)), color="#e74c3c")
rows.set_xlabel("# rows")
rows.set_xticks((0, 62.5, 125, 187.5, 250), labels=("0", "", "125", "", "250"))
rows.set_xlim((0, 250))
rows.set_ylabel("# tables")
rows.set_yticks((0, 40000, 80000, 120000), labels=("0", "", "", "120k"))
rows.set_ylim((0, 120000))

plt.savefig("../data/stats.pdf", bbox_inches="tight")
plt.show()
plt.clf()

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (12, 1.5)
plt.rcParams["font.size"] = 12
figure, (tabs, cols, rows) = plt.subplots(nrows=1, ncols=3, sharex=False, sharey=False)
figure.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.35, hspace=0.7)

MAX_TAB_COUNT = 10
tab_count = collections.Counter(stats["num_tables"])
tab_list = list(map(lambda x: (str(x[0]), x[1]), filter(lambda x: x[0] <= MAX_TAB_COUNT, sorted(tab_count.items())))) + [(f">{MAX_TAB_COUNT}", sum(map(lambda x: x[1], filter(lambda x: x[0] > MAX_TAB_COUNT, tab_count.items()))))]

x_lab, y = zip(*tab_list)
x = list(range(len(x_lab)))
tabs.bar(x, y, color="#e74c3c")
tabs.set_xlabel("# tables")
tabs.set_xticks(x, labels=x_lab, fontsize=10)
tabs.set_ylabel("# databases")
tabs.set_yticks((0, 3000, 6000, 9000), labels=("0", "", "", "9k"))
tabs.set_ylim((0, 9000))
tabs.xaxis.grid(False)

cols.hist(stats["num_cols"], bins=list(range(0, 101, 10)), color="#e74c3c")
cols.set_xlabel("# columns")
cols.set_xticks((2, 25, 50, 75, 100), labels=("2", "", "50", "", "100"))
cols.set_xlim((2, 100))
cols.set_ylabel("# tables")
cols.set_yticks((0, 40000, 80000, 120000), labels=("0", "", "", "120k"))
cols.set_ylim((0, 120000))

rows.hist(stats["num_rows"], bins=list(range(0, 201, 20)), color="#e74c3c")
rows.set_xlabel("# rows")
rows.set_xticks((1, 50, 100, 150, 200), labels=("1", "", "100", "", "200"))
rows.set_xlim((1, 200))
rows.set_ylabel("# tables")
rows.set_yticks((0, 40000, 80000, 120000), labels=("0", "", "", "120k"))
rows.set_ylim((0, 120000))

plt.savefig("../data/dimensions.pdf", bbox_inches="tight")
plt.show()
plt.clf()

In [ ]:
def ticks(max_v):
    return [10 ** (math.log10(max_v) / 4 * x) for x in range(5)]


def format_v(v):
    # if v % 1000000 == 0:
    #     return str(int(v / 1000000)) + "M"
    # if v % 1000 == 0:
    #     return str(int(v / 1000)) + "T"
    if int(v) == v:
        return str(v)

    return f"{v:.2f}"


def tick_labels(max_v):
    return [format_v(int(10 ** (math.log10(max_v) / 4 * x))) if x % 2 == 0 else "" for x in range(5)]

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (10, 1.5)
plt.rcParams["font.size"] = 14
figure, (tabs, cols, rows) = plt.subplots(nrows=1, ncols=3, sharex=False, sharey=False)
figure.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.4, hspace=0.7)

TABS_MAX_X = 100

num_tables_x = []
num_tables_y = []
curr_val = 0
for num, val in enumerate(sorted(stats["num_tables"])):
    while val >= curr_val:
        num_tables_x.append(curr_val)
        num_tables_y.append(num / len(stats["num_tables"]))
        curr_val += 1

while curr_val < TABS_MAX_X:
    num_tables_x.append(curr_val)
    num_tables_y.append(1)
    curr_val += 1

tabs.plot(num_tables_x, num_tables_y, color="#e74c3c")
tabs.set_xlabel("tables per DB [log]")
tabs.set_xscale("log")
tabs.set_xticks(ticks(TABS_MAX_X), labels=tick_labels(TABS_MAX_X))
tabs.set_xlim((1, TABS_MAX_X))
tabs.set_ylim((0, 1))
tabs.set_yticks((0, 0.25, 0.5, 0.75, 1), labels=("0", "", "0.5", "", "1"))

COLS_MAX_X = 1000

num_cols_x = []
num_cols_y = []
curr_val = 0
for num, val in enumerate(sorted(stats["num_cols"])):
    while val >= curr_val:
        num_cols_x.append(curr_val)
        num_cols_y.append(num / len(stats["num_cols"]))
        curr_val += 1
num_cols_x.append(curr_val)
num_cols_y.append(1)

while curr_val < COLS_MAX_X:
    num_cols_x.append(curr_val)
    num_cols_y.append(1)
    curr_val += 1

cols.plot(num_cols_x, num_cols_y, color="#e74c3c")
cols.set_xlabel("columns per table [log]")
cols.set_xscale("log")
cols.set_xticks(ticks(COLS_MAX_X), labels=tick_labels(COLS_MAX_X))
cols.set_xlim((1, COLS_MAX_X))
cols.set_ylim((0, 1))
cols.set_yticks((0, 0.25, 0.5, 0.75, 1), labels=("0", "", "0.5", "", "1"))

ROWS_MAX_X = 1000

num_rows_x = []
num_rows_y = []
curr_val = 0
for num, val in enumerate(sorted(stats["num_rows"])):
    while val >= curr_val:
        num_rows_x.append(curr_val)
        num_rows_y.append(num / len(stats["num_rows"]))
        curr_val += 1
num_rows_x.append(curr_val)
num_rows_y.append(1)

while curr_val < ROWS_MAX_X:
    num_rows_x.append(curr_val)
    num_rows_y.append(1)
    curr_val += 1

rows.plot(num_rows_x, num_rows_y, color="#e74c3c")
rows.set_xlabel("rows per table [log]")
rows.set_xscale("log")
rows.set_xticks(ticks(ROWS_MAX_X), labels=tick_labels(ROWS_MAX_X))
rows.set_xlim((1, ROWS_MAX_X))
rows.set_ylim((0, 1))
rows.set_yticks((0, 0.25, 0.5, 0.75, 1), labels=("0", "", "0.5", "", "1"))
# 
# rows.hist(stats["num_rows"], bins=list(10 ** (x / 2) for x in range(13)), color="black")
# rows.set_xlabel("number of rows")
# rows.set_xscale("log")
# rows.set_xticks((1, 31.622776601683793, 1000, 31622.776601683792, 1000000), labels=("1", "", "1T", "", "1M"))
# rows.set_xlim((1, 1000000))
# rows.set_yscale("log")
# rows.set_yticks((1,  31.622776601683793, 1000, 31622.776601683792, 1000000), labels=("1", "", "1T", "", "1M"))

#plt.savefig("../data/stats.pdf", bbox_inches="tight")
plt.show()
plt.clf()

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (12, 1.5)
plt.rcParams["font.size"] = 12
figure, (sparsity, dtypes, cnames) = plt.subplots(nrows=1, ncols=3, sharex=False, sharey=False)
figure.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.35, hspace=0.7)

sparsity.hist(stats["sparsities"], bins=list(x / 20 for x in range(0, 11, 1)), color="#e74c3c")
sparsity.set_xlabel("fraction of empty cells")
sparsity.set_xticks((0, 0.125, 0.25, 0.375, 0.5), labels=("0", "", "0.25", "", "0.5"))
sparsity.set_xlim((0, 0.5))
sparsity.set_ylabel("# tables")
sparsity.set_yticks((0, 30000, 60000, 90000), labels=("0", "", "", "90k"))
sparsity.set_ylim((0, 90000))

dtypes.bar([0, 1], [sum(stats["num_non_numerical"]), sum(stats["num_numerical"])], color=["#1abc9c", "#3498db"])
dtypes.set_xticks([0, 1], labels=("non-numerical", "numerical"))
dtypes.set_ylabel("# columns")
dtypes.set_yticks((0, 5000000, 10000000, 15000000), labels=("0", "", "", "15m"))
dtypes.set_ylim((0, 15000000))
dtypes.xaxis.grid(False)

cnames.bar([0, 3], [len(stats["tab_names"].keys()), len(stats["col_names"].keys())], color="#9b59b6", label="Wikidata")
cnames.bar([1, 4], [len(stats["llm_tab_names"].keys()), len(stats["llm_col_names"].keys())], color="#f39c12", label="Paraphrased")
cnames.set_xticks([0.5, 3.5], labels=("table names", "column names"))
cnames.set_ylabel("# unique names")
cnames.set_yticks((0, 30000, 60000, 90000), labels=("0", "", "", "90k"))
cnames.set_ylim((0, 90000))
cnames.xaxis.grid(False)
cnames.legend()

plt.savefig("../data/characteristics.pdf", bbox_inches="tight")
# .show()
plt.clf()